In [1]:
import pandas as pd
import numpy as np
import re
import emoji
import nltk
from matplotlib import pyplot as plt
from gensim.corpora import Dictionary
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
import pyLDAvis.gensim_models
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer
from HanTa import HanoverTagger as ht


nltk.download('wordnet')
nltk.download("stopwords")
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Topic Modelling

For topic modelling the same data set as for the sentiment analysis is used, which means that tweets not directly containing the hashtag, as well as retweets are removed. 

In [2]:
df = pd.read_csv ('data/tweets/IchBinHanna.csv')

In [3]:
#get the correct data
df['new_date'] = pd.to_datetime(df['created_at']).dt.strftime('%Y-%m-%d %H:%M:%S')
df = df.loc[(df['new_date'] > '2021-06-01 00:00:00') & (df['new_date'] <= '2021-09-30 23:59:59')]
df = df.loc[df['reference_type'] != 'retweeted']
df = df[df['text'].str.contains("#ichbinhanna", case = False)]
df['processed'] = df['text'].astype(str)
df = df.loc[(df['lang'] == 'de') | (df['lang'] == 'en')]

In [4]:
#clean the data (remove URLs, emojis and line breaks)
def rem_url(tweet):
     return " ".join(re.sub("([^0-9A-Za-zßäöü \t])|(\w+:\/\/\S+)", " ", tweet).split())
df['processed'] = df['processed'].apply(rem_url)
def rem_emojis(text):
    emojis = [x for x in text if x in emoji.UNICODE_EMOJI]
    cleaned = ' '.join([str for str in text.split() if not any(i in str for i in emojis)])
    return cleaned
df['processed'] = df['processed'].apply(lambda x: rem_emojis(x))
df['processed'] = df['text'].replace(r'\\n',  ' ', regex=True)
df['processed'] = df['processed'].astype(str)

In [5]:
#preprocessing (tokenization, stop word removal, stemming)
german_stop = set(stopwords.words('german'))
english_stop = set(stopwords.words('english'))
add_stop_all = ["ichbinhanna","#ichbinhanna", "hanna", "mehr", "innen", "#wisszeitvg", "#ichbinhannah", "@amreibahr", "amreibahr", "@bmf_bund","bmf_bund" "@drkeichhorn", "@bmbf_bund", "mehr", "innen", "schon", "gehen", "jahr","wissenschaft", "wissenschaftler", "kommen","academia", "academic", "year", "machen", "sagen", "sein","geben", "also", "werden", "german", "germany","gut", "haben", "geht", "gibt", "viele", "seit", "wäre"]
german_stop.update(set(add_stop_all))
english_stop.update(set(add_stop_all))
tweet_tokenizer = TweetTokenizer()
df['tokenized'] = df['processed'].apply(lambda x: tweet_tokenizer.tokenize(x.lower()))
df['tokenized'] = df[['tokenized','lang']].apply(lambda x: ' '.join([word for word in x['tokenized'] if word not in english_stop]).split() if x['lang'] == 'en' else ' '.join([word for word in x['tokenized'] if word not in german_stop]).split(),axis=1)
#remove all words consisting of 3 characters or less to shorten the dictionary of unique words
df['tokenized'] = df['tokenized'].apply(lambda x: [word for word in x if len(word) > 3])
lemmatizer = WordNetLemmatizer()
hannover = ht.HanoverTagger('morphmodel_ger.pgz')
df['tokenized'] = df[['tokenized','lang']].apply(lambda x: [lemmatizer.lemmatize(word).lower() for word in x['tokenized']] if x['lang'] == 'en' else [hannover.analyze(word)[0].lower() for word in x['tokenized']] ,axis=1)

In [6]:
def perform_LDA(tokens, topics=5, passes =5, alpha = 'symmetric', decay = 0.5):
    #create the dictionary of lemmatized tokens
    dic = Dictionary(tokens)
    print(len(dic))
    #remove low and high frequent terms
    dic.filter_extremes(no_below=2, no_above=.99)
    print(len(dic))
    #create the bag of words 
    corpus = [dic.doc2bow(d) for d in tokens]
    #build LDA model 
    LDA = LdaMulticore(corpus= corpus, num_topics=topics, id2word= dic, workers=12, passes=passes, alpha = alpha, decay = decay)
    words = [re.findall(r'"([^"]*)"',t[1]) for t in LDA.print_topics()]
    #create topics
    topics = [' '.join(t[0:10]) for t in words]

    for id, t in enumerate(topics): 
        print(f"------ Topic {id} ------")
        print(t, end="\n\n")
    # Compute Perplexity
    perplexity = LDA.log_perplexity(corpus)
    print('\nPerplexity: ', perplexity) 
    # Compute Coherence Score
    coherence_model = CoherenceModel(model=LDA, texts=tokens, 
                                   dictionary=dic, coherence='c_v')
    coherence_lda_model = coherence_model.get_coherence()
    print('\nCoherence Score: ', coherence_lda_model)
    return LDA

In [7]:
#perform topic modelling by applying LDA on the whole data set, and on the german/english only sets (guided by: https://towardsdatascience.com/twitter-topic-modeling-e0e3315b12e2)
#full data set
full_model = perform_LDA(df['tokenized'])

47228
15469
------ Topic 0 ------
gut finden lehre @anjakarliczek machen arbeit ganz arbeitsbedingung wissen forschung

------ Topic 1 ------
jahr befristet forschung arbeit vertrag gut promotion warum bekommen müssen

------ Topic 2 ------
gut @drkeichhorn #ichbinreyhan wichtig stelle wissen thread heute @sebastiankubon werden

------ Topic 3 ------
@anjakarliczek jahr system #hannaimbundestag @gew_bund problem machen aktuell perspektive immer

------ Topic 4 ------
arbeit hochschule gut stellen machen @anjakarliczek ganz immer heute befristet


Perplexity:  -8.701089009012147

Coherence Score:  0.18270245018946196


In [8]:
#only german tweets
df_ger = df.loc[df['lang'] == "de"]
df_en = df.loc[df['lang'] == "en"]
ger_model = perform_LDA(df_ger['tokenized'])

39811
12601
------ Topic 0 ------
#hannaimbundestag warum gut wichtig jahr werden @anjakarliczek hochschule deutsch wissen

------ Topic 1 ------
arbeit problem befristet jahr deutschland wissen stellen @anjakarliczek brauchen vertrag

------ Topic 2 ------
jahr befristet machen gut arbeit @gew_bund sehen immer ganz vertrag

------ Topic 3 ------
gut @anjakarliczek machen arbeitsbedingung @drkeichhorn uni system zeit forschung sprechen

------ Topic 4 ------
jahr arbeit forschung müssen stellen wissenschaftlich finden promotion heute werden


Perplexity:  -8.478811404902615

Coherence Score:  0.1422894176632658


In [9]:
#only english tweets
en_model = perform_LDA(df_en['tokenized'])

9256
3328
------ Topic 0 ------
contract @mahaelhissy many scholar position story work precarity academic issue

------ Topic 1 ------
work need year time scholar contract well system academic people

------ Topic 2 ------
researcher #ichbinreyhan working research condition permanent career precarious thread need

------ Topic 3 ------
thread system year contract many english university like career need

------ Topic 4 ------
system contract research work career student time university people change


Perplexity:  -7.812316900022977

Coherence Score:  0.22324389120296756


In [10]:
#get topics by month
df_june = df.loc[(df['new_date'] > '2021-06-01 00:00:00') & (df['new_date'] <= '2021-06-30 23:59:59')]
df_july = df.loc[(df['new_date'] > '2021-07-01 00:00:00') & (df['new_date'] <= '2021-07-31 23:59:59')]
df_august = df.loc[(df['new_date'] > '2021-08-01 00:00:00') & (df['new_date'] <= '2021-08-31 23:59:59')]
df_september = df.loc[(df['new_date'] > '2021-09-01 00:00:00') & (df['new_date'] <= '2021-09-30 23:59:59')]

In [11]:
#tweets from june
june_model = perform_LDA(df_june['tokenized'])

31199
10502
------ Topic 0 ------
jahr machen arbeit problem sehen forschung deutschland letzter universität hashtag

------ Topic 1 ------
@anjakarliczek #hannaimbundestag arbeit jahr hochschule befristet gut gerade aktuell brauchen

------ Topic 2 ------
gut arbeit jahr wissen vertrag forschung einfach haben befristet sagen

------ Topic 3 ------
jahr #hannaimbundestag stelle gut @anjakarliczek befristet system wissen eigentlich promotion

------ Topic 4 ------
thread system jahr #hannaimbundestag mensch @drkeichhorn neu @sebastiankubon befristet zeit


Perplexity:  -8.48554840120984

Coherence Score:  0.19214638929115802


In [12]:
#tweets from july
july_model = perform_LDA(df_july['tokenized'])

16767
5780
------ Topic 0 ------
deutsch @anjakarliczek hochschule forschung haben #hannabeidergew @drkeichhorn eigentlich university gerade

------ Topic 1 ------
#ichbinreyhan machen gut system @gew_bund wichtig danken problem jahr werden

------ Topic 2 ------
gut arbeit #ichbinreyhan @drkeichhorn stellen dafür jahr immer @anjakarliczek befristet

------ Topic 3 ------
jahr @anjakarliczek sehen stellen brauchen forschung beschäftigt stelle ganz @gew_bund

------ Topic 4 ------
#ichbinreyhan müssen @drkeichhorn befristet arbeit problem #hannabeidergew brauchen wissen @tagesthemen


Perplexity:  -8.403935488894236

Coherence Score:  0.25290280229756024


In [13]:
#tweets from august
august_model = perform_LDA(df_august['tokenized'])

9918
3233
------ Topic 0 ------
stellen #ichbinreyhan finden frage @drkeichhorn problem @karolinedoering bleiben system arbeit

------ Topic 1 ------
#ichbinreyhan arbeit machen heute sehen werden immer hochschule system warum

------ Topic 2 ------
gut @gew_bund zeit @anjakarliczek müssen @drkeichhorn wirklich #dauerstell neu einfach

------ Topic 3 ------
sprechen jahr aktuell denken gut arbeit #ichbinreyhan immer @drkeichhorn folge

------ Topic 4 ------
#ichbinreyhan wissen #waspostdocswoll ganz kommen akademisch gut #dauerstell arbeit system


Perplexity:  -8.061990560067759

Coherence Score:  0.30948090042844856


In [14]:
#tweets from september
september_model = perform_LDA(df_september['tokenized'])

9231
3015
------ Topic 0 ------
gut @gew_bund lehre immer jahr #ichbinreyhan promotion sehen heute forschung

------ Topic 1 ------
arbeit #ichbinreyhan neu stellen befristet vertrag heute jahr #frististfrust gut

------ Topic 2 ------
@gew_bund jahr ganz monat hochschule vertrag #ichbinreyhan aktuell #hannainzahlen #dauerstell

------ Topic 3 ------
gut #ichbinreyhan machen arbeit haben thema wissenschaftlich sprechen wählen heute

------ Topic 4 ------
#ichbinreyhan problem dürfen forschung system uni #btw21 wichtig neu groß


Perplexity:  -8.0318888456595

Coherence Score:  0.3422135791985527


In [15]:
def visualize_topics(tokens):
    #Visualization of topic distance 
    vis_dic = Dictionary(tokens)
    vis_dic.filter_extremes(no_below=2, no_above=.99)
    #create the bag of words 
    vis_corpus = [vis_dic.doc2bow(d) for d in tokens]
    #build LDA model a 
    vis_LDA = LdaMulticore(corpus= vis_corpus, num_topics=5, id2word= vis_dic, workers=12, passes=5, random_state = 1)
    pyLDAvis.enable_notebook()
    return pyLDAvis.gensim_models.prepare(vis_LDA, vis_corpus, vis_dic)
def visualize_model(model, tokens):
    vis_dic = Dictionary(tokens)
    vis_dic.filter_extremes(no_below=2, no_above=.99)
    vis_corpus = [vis_dic.doc2bow(d) for d in tokens]
    pyLDAvis.enable_notebook()
    return pyLDAvis.gensim_models.prepare(model, vis_corpus, vis_dic)

In [16]:
visualize_topics(df['tokenized'])

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.074733  0.008764       1        1  26.116776
3     -0.038672  0.004456       2        1  23.102126
0     -0.014847 -0.006811       3        1  19.613206
1      0.072514  0.064575       4        1  15.878367
4      0.055738 -0.070984       5        1  15.289525, topic_info=          Term         Freq        Total Category  logprob  loglift
150       jahr  1385.000000  1385.000000  Default  30.0000  30.0000
286    problem   671.000000   671.000000  Default  29.0000  29.0000
626    letzter   410.000000   410.000000  Default  28.0000  28.0000
839    vertrag   589.000000   589.000000  Default  27.0000  27.0000
205  befristet   882.000000   882.000000  Default  26.0000  26.0000
..         ...          ...          ...      ...      ...      ...
539     wissen    83.059754   687.675368   Topic5  -6.0437  -0.2358
161     arbeit    90.033136   987.405930   Topic5  -5.9631  -0.5169
168     machen    83.543876   783.303812   Topic5  -6.0379  -0.3601
619      frage    67.610135   436.035257   Topic5  -6.2496   0.0140
930      immer    69.023112   712.414059   Topic5  -6.2289  -0.4562

[491 rows x 6 columns], token_table=       Topic      Freq                      Term
term                                            
480        1  0.179559           #95vswisszeitvg
480        2  0.113951           #95vswisszeitvg
480        3  0.341853           #95vswisszeitvg
480        4  0.262432           #95vswisszeitvg
480        5  0.103592           #95vswisszeitvg
...      ...       ...                       ...
14956      1  0.965334                     ️ende
14957      1  0.874542  ️erwerbsarbeitslosigkeit
14958      1  0.865284                   ️kinder
14960      1  0.888122               ️stipendium
14960      2  0.080738               ️stipendium

[1291 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 1, 2, 5])

In [17]:
#get optimal number of topics for each (sub)set
def compute_coherence_values_topics(tokens, limit=10, start=2, step=1):
    
    dic = Dictionary(tokens)
    dic.filter_extremes(no_below=2, no_above=.99)
    corpus = [dic.doc2bow(d) for d in tokens]
    coherence_values_topic = []
    model_list_topic = []
    for num_topics in range(start, limit, step):
        model = LdaMulticore(corpus=corpus, num_topics=num_topics, id2word=dic, random_state = 1)
        model_list_topic.append(num_topics)
        coherencemodel = CoherenceModel(model=model, texts=tokens, dictionary=dic, coherence='c_v')
        coherence_values_topic.append(coherencemodel.get_coherence())

    print(model_list_topic, coherence_values_topic)
    
def compute_coherence_values_passes(tokens,num_topics):
    
    passes = [5,10,15,20]
    dic = Dictionary(tokens)
    dic.filter_extremes(no_below=2, no_above=.99)
    corpus = [dic.doc2bow(d) for d in tokens]
    coherence_values_passes = []
    model_list_passes = []
    for num_pass in passes:
        model = LdaMulticore(corpus=corpus, num_topics=num_topics, id2word=dic, passes = num_pass, random_state = 1)
        model_list_passes.append(num_pass)
        coherencemodel = CoherenceModel(model=model, texts=tokens, dictionary=dic, coherence='c_v')
        coherence_values_passes.append(coherencemodel.get_coherence())

    print(model_list_passes, coherence_values_passes)
    
def compute_coherence_values_alpha(tokens,num_topics, passes):
    
    alpha = ['symmetric','asymmetric']
    dic = Dictionary(tokens)
    dic.filter_extremes(no_below=2, no_above=.99)
    corpus = [dic.doc2bow(d) for d in tokens]
    coherence_values_alpha = []
    model_list_alpha = []
    for a in alpha:
        model = LdaMulticore(corpus=corpus, num_topics=num_topics, id2word=dic, passes = passes, alpha = a, random_state = 1)
        model_list_alpha.append(a)
        coherencemodel = CoherenceModel(model=model, texts=tokens, dictionary=dic, coherence='c_v')
        coherence_values_alpha.append(coherencemodel.get_coherence())

    print(model_list_alpha, coherence_values_alpha)
    
def compute_coherence_values_decay(tokens,num_topics, passes, alpha):
    
    decay = [0.5,0.7,0.9]
    dic = Dictionary(tokens)
    dic.filter_extremes(no_below=2, no_above=.99)
    corpus = [dic.doc2bow(d) for d in tokens]
    coherence_values_decay = []
    model_list_decay = []
    for d in decay:
        model = LdaMulticore(corpus=corpus, num_topics=num_topics, id2word=dic, passes = passes, alpha = alpha, random_state = 1, decay =d)
        model_list_decay.append(d)
        coherencemodel = CoherenceModel(model=model, texts=tokens, dictionary=dic, coherence='c_v')
        coherence_values_decay.append(coherencemodel.get_coherence())

    print(model_list_decay, coherence_values_decay)

In [18]:
#full data
compute_coherence_values_topics(df['tokenized'])
#english data
compute_coherence_values_topics(df_en['tokenized'])
#german data
compute_coherence_values_topics(df_ger['tokenized'])
#june data
compute_coherence_values_topics(df_june['tokenized'])
#july data
compute_coherence_values_topics(df_july['tokenized'])
#august data
compute_coherence_values_topics(df_august['tokenized'])
#september data
compute_coherence_values_topics(df_september['tokenized'])

[2, 3, 4, 5, 6, 7, 8, 9] [0.10456537301003273, 0.10933232716209958, 0.13983522052814684, 0.14401792967681504, 0.16532023615923622, 0.16484462935964542, 0.16624047616664991, 0.18046979098848526]
[2, 3, 4, 5, 6, 7, 8, 9] [0.1713934940401889, 0.1926387289590242, 0.20639512438015184, 0.22768174688364207, 0.2288697700012765, 0.23042403767341899, 0.2391635468428444, 0.2387414925840346]
[2, 3, 4, 5, 6, 7, 8, 9] [0.08968424088169547, 0.11143055442979366, 0.13090652216222032, 0.16296584062709765, 0.1683182159697678, 0.18246422016127548, 0.17465228169757455, 0.18142981907986136]
[2, 3, 4, 5, 6, 7, 8, 9] [0.10305715932532085, 0.12963539272547195, 0.13969566033120212, 0.17104263228258018, 0.1684735785092272, 0.20518778318038336, 0.1944392273439256, 0.19994246539845353]
[2, 3, 4, 5, 6, 7, 8, 9] [0.1596968713141924, 0.18342441323817812, 0.2316848091945012, 0.23033421945001115, 0.21452232034663432, 0.21691375176596392, 0.22270465611598422, 0.20162710324766536]
[2, 3, 4, 5, 6, 7, 8, 9] [0.218210534108

In [19]:
#full data
compute_coherence_values_passes(df['tokenized'], 6)
#english data
compute_coherence_values_passes(df_en['tokenized'],8)
#german data
compute_coherence_values_passes(df_ger['tokenized'],8)
#june data
compute_coherence_values_passes(df_june['tokenized'],8)
#july data
compute_coherence_values_passes(df_july['tokenized'],9)
#august data
compute_coherence_values_passes(df_august['tokenized'],9)
#september data
compute_coherence_values_passes(df_september['tokenized'],8)

[5, 10, 15, 20] [0.2232769219268248, 0.2690152610411907, 0.3228316499418125, 0.3268002840348955]
[5, 10, 15, 20] [0.24915991509879745, 0.2612628445116889, 0.26380476019690424, 0.271753624443677]
[5, 10, 15, 20] [0.1939851769211308, 0.21568031194880405, 0.2322086647919751, 0.23678522027750526]
[5, 10, 15, 20] [0.24474511422358788, 0.2869218881627225, 0.3260379225338824, 0.3397137101961771]
[5, 10, 15, 20] [0.2554959019382797, 0.24402870026977952, 0.2534215279594163, 0.2621248722547103]
[5, 10, 15, 20] [0.2824516647033783, 0.29574032006188355, 0.29849572880663794, 0.30912229640979666]
[5, 10, 15, 20] [0.32174582869474205, 0.31718306381680816, 0.3142800230028888, 0.32047759338204074]


In [20]:
#full data
compute_coherence_values_alpha(df['tokenized'], 6,20)
#english data
compute_coherence_values_alpha(df_en['tokenized'],8,20)
#german data
compute_coherence_values_alpha(df_ger['tokenized'],8,20)
#june data
compute_coherence_values_alpha(df_june['tokenized'],8,20)
#july data
compute_coherence_values_alpha(df_july['tokenized'],9,20)
#august data
compute_coherence_values_alpha(df_august['tokenized'],9,20)
#september data
compute_coherence_values_alpha(df_september['tokenized'],8,5)

['symmetric', 'asymmetric'] [0.3268002840348955, 0.3809749124568735]
['symmetric', 'asymmetric'] [0.271753624443677, 0.2538859498789313]
['symmetric', 'asymmetric'] [0.23701694936258688, 0.20978025739110578]
['symmetric', 'asymmetric'] [0.34050205707609915, 0.38977301977262047]
['symmetric', 'asymmetric'] [0.2621248722547103, 0.35178133343358603]
['symmetric', 'asymmetric'] [0.30912229640979666, 0.36407715047510764]
['symmetric', 'asymmetric'] [0.32174582869474205, 0.3279922890145045]


In [21]:
#full data
compute_coherence_values_decay(df['tokenized'], 6,20,'asymmetric')
#english data
compute_coherence_values_decay(df_en['tokenized'],8,20,'symmetric')
#german data
compute_coherence_values_decay(df_ger['tokenized'],8,20,'symmetric')
#june data
compute_coherence_values_decay(df_june['tokenized'],8,20,'asymmetric')
#july data
compute_coherence_values_decay(df_july['tokenized'],9,20,'asymmetric')
#august data
compute_coherence_values_decay(df_august['tokenized'],9,20,'asymmetric')
#september data
compute_coherence_values_decay(df_september['tokenized'],8,5,'asymmetric')

[0.5, 0.7, 0.9] [0.3809749124568735, 0.2963046085250385, 0.26718107317632733]
[0.5, 0.7, 0.9] [0.271753624443677, 0.26643195527369967, 0.24938472551264299]
[0.5, 0.7, 0.9] [0.2385812567691823, 0.21842055543396516, 0.19684653149208034]
[0.5, 0.7, 0.9] [0.3916118752563964, 0.3606002405298114, 0.29765623633544497]
[0.5, 0.7, 0.9] [0.35178133343358603, 0.31524882276942967, 0.30781519531133594]
[0.5, 0.7, 0.9] [0.36407715047510764, 0.34814232942355183, 0.3292749688157396]
[0.5, 0.7, 0.9] [0.3279922890145045, 0.3288853538609816, 0.33193391268330086]


In [22]:
#get the optimal models
optimal_full = perform_LDA(df['tokenized'],6,20,'asymmetric',0.5)
optimal_en = perform_LDA(df_en['tokenized'],8,20,'symmetric',0.5)
optimal_ger = perform_LDA(df_ger['tokenized'],8,20,'symmetric',0.5)
optimal_june = perform_LDA(df_june['tokenized'],8,20,'asymmetric',0.5)
optimal_july = perform_LDA(df_july['tokenized'],9,20,'asymmetric',0.5)
optimal_august = perform_LDA(df_august['tokenized'],9,20,'asymmetric',0.5)
optimal_september = perform_LDA(df_september['tokenized'],8,5,'asymmetric',0.9)

47228
15469
------ Topic 0 ------
@anjakarliczek arbeit gut forschung #hannaimbundestag wissen machen finden jahr hochschule

------ Topic 1 ------
jahr gut @drkeichhorn immer promotion haben @sebastiankubon befristet stellen stelle

------ Topic 2 ------
system contract research work year career position university researcher many

------ Topic 3 ------
jahr vertrag befristet kind monat @anjakarliczek arbeit #hannaimbundestag werden zeit

------ Topic 4 ------
erster workshop #wissenschaftler bekommen #karrieren schreiben #wissenschaftlerinnen sprechen wissenschaftlich story

------ Topic 5 ------
befristet universität promotion wissenschaftlich problem deutschland machen gut ganz promovieren


Perplexity:  -8.467444119864194

Coherence Score:  0.3548593330998684
9256
3328
------ Topic 0 ------
system science many tweet #ichbinreyhan think contract scientist like time

------ Topic 1 ------
thread system academic researcher permanent research work position university contract

------ 

In [23]:
visualize_model(optimal_full, df['tokenized'])

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.128425  0.046758       1        1  49.991664
1      0.128941  0.052662       2        1  22.056500
2     -0.348707  0.067497       3        1  12.540024
3      0.091165  0.053877       4        1   9.040611
4     -0.034315 -0.195405       5        1   3.402258
5      0.034491 -0.025388       6        1   2.968942, topic_info=                  Term         Freq        Total Category  logprob  loglift
150               jahr  1339.000000  1339.000000  Default  30.0000  30.0000
205          befristet   841.000000   841.000000  Default  29.0000  29.0000
839            vertrag   551.000000   551.000000  Default  28.0000  28.0000
171          promotion   528.000000   528.000000  Default  27.0000  27.0000
601        universität   343.000000   343.000000  Default  26.0000  26.0000
...                ...          ...          ...      ...      ...      ...
227   arbeitsbedingung    14.908565   439.039413   Topic6  -6.1224   0.1343
1599            gelten    12.093252   154.779467   Topic6  -6.3317   0.9676
29              kommen    13.143047   370.238553   Topic6  -6.2485   0.1787
543         befristung    13.013706   379.603228   Topic6  -6.2583   0.1438
930              immer    12.367196   707.671399   Topic6  -6.3093  -0.5300

[472 rows x 6 columns], token_table=       Topic      Freq              Term
term                                    
4577       1  0.028629         #baerbock
4577       2  0.458068         #baerbock
4577       4  0.229034         #baerbock
4577       6  0.286293         #baerbock
7379       1  0.030791           #berlhg
...      ...       ...               ...
14954      5  0.074003  ️arbeitsverträge
14956      3  0.084937             ️ende
14956      4  0.849367             ️ende
14960      3  0.092108       ️stipendium
14960      4  0.921083       ️stipendium

[1202 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6])

In [24]:
visualize_model(optimal_en, df_en['tokenized'])

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.045706 -0.091982       1        1  17.661399
2     -0.041990  0.036092       2        1  14.714597
3     -0.017371 -0.033274       3        1  14.181234
1     -0.023251 -0.113792       4        1  13.189237
6     -0.013347  0.087460       5        1  12.032332
4     -0.041428  0.101212       6        1  10.095672
0     -0.021351  0.015380       7        1   9.561292
5      0.204444 -0.001094       8        1   8.564238, topic_info=          Term        Freq       Total Category  logprob  loglift
69      thread  151.000000  151.000000  Default  30.0000  30.0000
820      story   74.000000   74.000000  Default  29.0000  29.0000
913  @gew_bund   41.000000   41.000000  Default  28.0000  28.0000
782      share   56.000000   56.000000  Default  27.0000  27.0000
591   contract  233.000000  233.000000  Default  26.0000  26.0000
..         ...         ...         ...      ...      ...      ...
90    position   12.764036  145.572536   Topic8  -5.2911   0.0235
545     please   10.503030   46.301279   Topic8  -5.4860   0.9741
184       need   11.023581  133.912654   Topic8  -5.4376  -0.0396
625      check   10.343114   45.978056   Topic8  -5.5014   0.9657
138     system   10.535566  237.256888   Topic8  -5.4829  -0.6568

[626 rows x 6 columns], token_table=      Topic      Freq             Term
term                                  
1605      1  0.117875  #95vswisszeitvg
1605      3  0.058938  #95vswisszeitvg
1605      4  0.383095  #95vswisszeitvg
1605      6  0.058938  #95vswisszeitvg
1605      7  0.294688  #95vswisszeitvg
...     ...       ...              ...
80        4  0.254233            young
80        5  0.136895            young
80        6  0.058669            young
80        7  0.156451            young
80        8  0.019556            young

[1518 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 3, 4, 2, 7, 5, 1, 6])

In [25]:
visualize_model(optimal_ger, df_ger['tokenized'])

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.115630  0.028373       1        1  17.187162
2     -0.066111 -0.009314       2        1  13.142311
3      0.052043 -0.037631       3        1  12.800576
0     -0.075305  0.017669       4        1  12.644941
5      0.027008 -0.006139       5        1  12.315260
1     -0.002937 -0.069908       6        1  11.669366
4      0.095816 -0.059358       7        1  10.655788
6      0.085116  0.136308       8        1   9.584597, topic_info=                Term         Freq        Total Category  logprob  loglift
78            stunde   244.000000   244.000000  Default  30.0000  30.0000
122             jahr  1352.000000  1352.000000  Default  29.0000  29.0000
304        @gew_bund   413.000000   413.000000  Default  28.0000  28.0000
506  @sebastiankubon   284.000000   284.000000  Default  27.0000  27.0000
192    #ichbinreyhan   511.000000   511.000000  Default  26.0000  26.0000
..               ...          ...          ...      ...      ...      ...
133           arbeit    55.147338   976.756522   Topic8  -5.8409  -0.5292
851            immer    53.053497   705.690153   Topic8  -5.8796  -0.2429
254          problem    51.050394   571.370989   Topic8  -5.9181  -0.0702
81            werden    51.300073   621.862503   Topic8  -5.9132  -0.1500
32            müssen    49.558079   610.626667   Topic8  -5.9477  -0.1663

[677 rows x 6 columns], token_table=       Topic      Freq                      Term
term                                            
438        1  0.119246           #95vswisszeitvg
438        2  0.015387           #95vswisszeitvg
438        3  0.073086           #95vswisszeitvg
438        4  0.061546           #95vswisszeitvg
438        5  0.315424           #95vswisszeitvg
...      ...       ...                       ...
12129      4  0.918461          ️arbeitsverträge
12131      4  0.896909                     ️ende
12132      4  0.953994  ️erwerbsarbeitslosigkeit
12133      4  0.942653                   ️kinder
12135      4  0.883309               ️stipendium

[2096 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 3, 4, 1, 6, 2, 5, 7])

In [26]:
visualize_model(optimal_june, df_june['tokenized'])

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.065243  0.059849       1        1  27.747352
1     -0.071915  0.031689       2        1  17.927497
3     -0.085318 -0.093648       3        1  15.234105
7     -0.082293 -0.056761       4        1   9.130295
4     -0.049379  0.171539       5        1   8.817426
2     -0.027120 -0.096405       6        1   8.597560
6      0.422432 -0.002784       7        1   7.799005
5     -0.041164 -0.013478       8        1   4.746761, topic_info=                  Term        Freq       Total Category  logprob  loglift
303               jahr  972.000000  972.000000  Default  30.0000  30.0000
362             stunde  198.000000  198.000000  Default  29.0000  29.0000
138          befristet  572.000000  572.000000  Default  28.0000  28.0000
341  #hannaimbundestag  673.000000  673.000000  Default  27.0000  27.0000
468           contract  151.000000  151.000000  Default  26.0000  26.0000
..                 ...         ...         ...      ...      ...      ...
143               zeit   19.633991  323.297859   Topic8  -5.7501   0.2464
160                neu   17.271047  225.133471   Topic8  -5.8783   0.4800
23             deutsch   17.436338  248.461708   Topic8  -5.8688   0.3910
4             brauchen   17.684100  308.674340   Topic8  -5.8547   0.1881
435              haben   16.142861  341.177916   Topic8  -5.9459  -0.0032

[642 rows x 6 columns], token_table=      Topic      Freq             Term
term                                  
168       1  0.450693  #95vswisszeitvg
168       2  0.113911  #95vswisszeitvg
168       3  0.123817  #95vswisszeitvg
168       4  0.034669  #95vswisszeitvg
168       5  0.099053  #95vswisszeitvg
...     ...       ...              ...
1769      4  0.793597     überarbeitet
2745      2  0.260271      überflüssig
2745      6  0.086757      überflüssig
2745      8  0.607299      überflüssig
866       3  0.926921        überhöhen

[1641 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 4, 8, 5, 3, 7, 6])

In [27]:
visualize_model(optimal_july, df_july['tokenized'])

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.103981 -0.017372       1        1  31.065736
1      0.128593 -0.034564       2        1  21.225601
2     -0.143233  0.177611       3        1  12.164756
4      0.029399  0.000498       4        1   8.282381
3     -0.029191  0.010917       5        1   7.716610
5      0.054152  0.013221       6        1   6.359174
8     -0.231278 -0.138490       7        1   4.841644
6      0.075885 -0.009541       8        1   4.612114
7      0.011693 -0.002280       9        1   3.731984, topic_info=                 Term        Freq       Total Category  logprob  loglift
191         befristet  144.000000  144.000000  Default  30.0000  30.0000
10               work   63.000000   63.000000  Default  29.0000  29.0000
38                gut  245.000000  245.000000  Default  28.0000  28.0000
315         @gew_bund  134.000000  134.000000  Default  27.0000  27.0000
4755  #hannabeidergew   90.000000   90.000000  Default  26.0000  26.0000
...               ...         ...         ...      ...      ...      ...
1198             frau    4.832882   72.998994   Topic9  -5.8931   0.5732
410            können    4.572420   50.070100   Topic9  -5.9485   0.8948
252           wichtig    4.764884  112.482899   Topic9  -5.9073   0.1267
573             warum    4.528954   85.648390   Topic9  -5.9580   0.3485
158             sehen    4.485799  129.702861   Topic9  -5.9676  -0.0761

[691 rows x 6 columns], token_table=      Topic      Freq                          Term
term                                               
146       3  0.820410                 #95wisspausen
146       4  0.082041                 #95wisspausen
146       8  0.082041                 #95wisspausen
455       1  0.275958  #acertaindegreeofflexibility
455       2  0.183972  #acertaindegreeofflexibility
...     ...       ...                           ...
1906      8  0.233274                    österreich
1906      9  0.699821                    österreich
4252      3  0.166790                          üben
4252      5  0.667162                          üben
5474      4  0.881696                      übergang

[1741 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 5, 4, 6, 9, 7, 8])

In [28]:
visualize_model(optimal_august, df_august['tokenized'])

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.076360 -0.025694       1        1  33.294063
1     -0.038855 -0.062206       2        1  18.978360
2     -0.111245 -0.028654       3        1  14.916147
3     -0.013379  0.064823       4        1   8.508173
5      0.003073  0.195722       5        1   6.127017
4      0.194281 -0.033725       6        1   5.961348
7      0.045649 -0.011293       7        1   4.354799
6     -0.018925 -0.038253       8        1   4.202610
8      0.015761 -0.060720       9        1   3.657483, topic_info=               Term       Freq      Total Category  logprob  loglift
39      #dauerstell  42.000000  42.000000  Default  30.0000  30.0000
403  @anjakarliczek  75.000000  75.000000  Default  29.0000  29.0000
42        @gew_bund  50.000000  50.000000  Default  28.0000  28.0000
792    @mahaelhissy  35.000000  35.000000  Default  27.0000  27.0000
38          wichtig  54.000000  54.000000  Default  26.0000  26.0000
..              ...        ...        ...      ...      ...      ...
1           deutsch   2.912907  38.769501   Topic9  -5.5617   0.7199
620          machen   2.957239  74.031868   Topic9  -5.5466   0.0882
235          sorgen   2.527300  16.457782   Topic9  -5.7037   1.4347
174          arbeit   2.956274  98.413500   Topic9  -5.5469  -0.1969
38          wichtig   2.598561  54.741271   Topic9  -5.6759   0.2607

[605 rows x 6 columns], token_table=      Topic      Freq                   Term
term                                        
143       6  0.741506  #academicmentalhealth
1762      5  0.927945              #afghanen
2082      2  0.850659   #anschlussverwendung
2082      6  0.141777   #anschlussverwendung
1846      8  0.866340  #arbeitszeiterfassung
...     ...       ...                    ...
1167      2  0.905134                zweiter
1277      3  0.854336                zwingen
1401      2  0.071675                überall
1401      3  0.860095                überall
1401      9  0.071675                überall

[1334 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 6, 5, 8, 7, 9])

In [29]:
visualize_model(optimal_september, df_september['tokenized'])

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.006990 -0.003442       1        1  38.320302
1      0.036746 -0.052649       2        1  17.424528
2      0.034016 -0.090259       3        1  13.562829
3      0.044655  0.145432       4        1   9.528521
5     -0.157634 -0.005701       5        1   8.522519
4      0.057152 -0.017027       6        1   5.772693
6     -0.055278  0.024591       7        1   3.779217
7      0.033355 -0.000944       8        1   3.089390, topic_info=                Term        Freq       Total Category  logprob  loglift
51           vertrag   69.000000   69.000000  Default  30.0000  30.0000
186            monat   52.000000   52.000000  Default  29.0000  29.0000
10               neu   78.000000   78.000000  Default  28.0000  28.0000
1613  #hannainzahlen   46.000000   46.000000  Default  27.0000  27.0000
22     #ichbinreyhan  147.000000  147.000000  Default  26.0000  26.0000
...              ...         ...         ...      ...      ...      ...
148           wissen    2.141190   49.091632   Topic8  -5.5805   0.3449
215             ganz    2.062588   42.482878   Topic8  -5.6179   0.4521
451              uni    1.973506   41.243492   Topic8  -5.6621   0.4375
1898           track    1.702561   14.918399   Topic8  -5.8097   1.3067
191          @_verdi    1.737621   20.317106   Topic8  -5.7894   1.0182

[577 rows x 6 columns], token_table=      Topic      Freq             Term
term                                  
77        1  0.189966  #95vswisszeitvg
77        2  0.227959  #95vswisszeitvg
77        3  0.075986  #95vswisszeitvg
77        4  0.303946  #95vswisszeitvg
77        6  0.113980  #95vswisszeitvg
...     ...       ...              ...
2412      7  0.273649          ️kinder
2740      5  0.783915         ️mentale
2414      1  0.096780      ️stipendium
2414      4  0.096780      ️stipendium
2414      5  0.774240      ️stipendium

[1358 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 6, 5, 7, 8])